In [1]:
!pip install findspark
!pip install pyspark
!pip install pyarrow
!pip install fastparquet
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5b9a2d2aacadbc83b5b9e500592455c57f11c58d83df81310060fbf0b71f2d4c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.1 MB/s eta 0:00:00


In [2]:
# Importing our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [3]:
# Installing Java and Spark and starting a SparkSession
import os

spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Installing Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Setting Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Starting a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,756 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,371 kB]
Get:13 http://security.ubun

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [5]:
spark

In [6]:
# 1. Reading in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://heart-2022-no-nans.s3.us-east-2.amazonaws.com/heart_2022_no_nans.csv"
spark.sparkContext.addFile(url)
heart_disease_df = spark.read.csv(SparkFiles.get("heart_2022_no_nans.csv"), sep=",", header=True)
heart_disease_df.show()

+-------+------+-------------+------------------+----------------+--------------------+------------------+----------+--------------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+--------------+-----------------+-----+---------------+----------+------------+-------------+--------------------+----------------+--------+
|  State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities|SleepHours|        RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|DifficultyDress

In [7]:
# 2. Creating a temporary view of the DataFrame.
heart_disease_df.createOrReplaceTempView("heart_disease_temp_view")

In [8]:
# Describing the DataFrame
heart_disease_df.describe()

DataFrame[summary: string, State: string, Sex: string, GeneralHealth: string, PhysicalHealthDays: string, MentalHealthDays: string, LastCheckupTime: string, PhysicalActivities: string, SleepHours: string, RemovedTeeth: string, HadHeartAttack: string, HadAngina: string, HadStroke: string, HadAsthma: string, HadSkinCancer: string, HadCOPD: string, HadDepressiveDisorder: string, HadKidneyDisease: string, HadArthritis: string, HadDiabetes: string, DeafOrHardOfHearing: string, BlindOrVisionDifficulty: string, DifficultyConcentrating: string, DifficultyWalking: string, DifficultyDressingBathing: string, DifficultyErrands: string, SmokerStatus: string, ECigaretteUsage: string, ChestScan: string, RaceEthnicityCategory: string, AgeCategory: string, HeightInMeters: string, WeightInKilograms: string, BMI: string, AlcoholDrinkers: string, HIVTesting: string, FluVaxLast12: string, PneumoVaxEver: string, TetanusLast10Tdap: string, HighRiskLastYear: string, CovidPos: string]

In [9]:
# Showing schema for the DataFrame
heart_disease_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- GeneralHealth: string (nullable = true)
 |-- PhysicalHealthDays: string (nullable = true)
 |-- MentalHealthDays: string (nullable = true)
 |-- LastCheckupTime: string (nullable = true)
 |-- PhysicalActivities: string (nullable = true)
 |-- SleepHours: string (nullable = true)
 |-- RemovedTeeth: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- HadAngina: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- HadCOPD: string (nullable = true)
 |-- HadDepressiveDisorder: string (nullable = true)
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadArthritis: string (nullable = true)
 |-- HadDiabetes: string (nullable = true)
 |-- DeafOrHardOfHearing: string (nullable = true)
 |-- BlindOrVisionDifficulty: string (nullable = true)
 |-- DifficultyConcentrating: string (nullable = t

In [10]:
# Replacing "Yes" with 1 and "No" with 0 in the specified columns

from pyspark.sql.functions import when

# Replacing specified columns in the DataFrame
columns_to_replace = ["HadHeartAttack", "HadAngina", "HadStroke", "HadAsthma", "HadSkinCancer",
                      "HadCOPD", "HadDepressiveDisorder", "HadKidneyDisease", "HadArthritis",
                      "HadDiabetes", "DeafOrHardOfHearing", "BlindOrVisionDifficulty",
                      "DifficultyConcentrating", "DifficultyWalking", "DifficultyDressingBathing",
                      "DifficultyErrands", "AlcoholDrinkers", "HIVTesting", "FluVaxLast12",
                      "PneumoVaxEver", "TetanusLast10Tdap", "HighRiskLastYear", "CovidPos"]

for column in columns_to_replace:
    heart_disease_df = heart_disease_df.withColumn(column, when(heart_disease_df[column] == "Yes", 1).otherwise(0))

# Showing the DataFrame with replaced values
heart_disease_df.show()


+-------+------+-------------+------------------+----------------+--------------------+------------------+----------+--------------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+--------------+-----------------+-----+---------------+----------+------------+-------------+-----------------+----------------+--------+
|  State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities|SleepHours|        RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|DifficultyDressing

In [11]:
# Cleaning data by searching for missing values within the DataFrame

from pyspark.sql.functions import col

# List columns with missing values
columns_with_missing_values = [col_name for col_name in heart_disease_df.columns if heart_disease_df.where(col(col_name).isNull()).count() > 0]

# Show columns with missing values
print("Columns with missing values:")
if columns_with_missing_values:
    print(", ".join(columns_with_missing_values))
else:
    print("No columns have missing values.")



Columns with missing values:
No columns have missing values.


In [12]:
# Looking for duplicate rows in the Dataset and Removing them

# Removing duplicate rows from the DataFrame
cleaned_df = heart_disease_df.dropDuplicates()

# Checking the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:", (cleaned_df.count(), len(cleaned_df.columns)))


Shape of cleaned DataFrame: (246004, 40)


In [13]:
# 3. Are males or females more likely to be at risk for heart disease?

from pyspark.sql import functions as F

# Grouping by 'Sex' column and count the occurrences
count_by_sex = heart_disease_df.groupBy('Sex').count()

# Showing the count for each sex
count_by_sex.show()

# Grouping by 'Sex' column and sum the occurrences of 'HadHeartAttack'
risk_by_sex = heart_disease_df.groupBy('Sex').agg(F.sum('HadHeartAttack').alias('TotalHeartAttacks'))

# Showing the total heart attacks for each sex
risk_by_sex.show()

# Determining which gender has a higher total heart attacks
male_heart_attacks = risk_by_sex.filter(risk_by_sex['Sex'] == 'Male').select('TotalHeartAttacks').collect()[0][0]
female_heart_attacks = risk_by_sex.filter(risk_by_sex['Sex'] == 'Female').select('TotalHeartAttacks').collect()[0][0]

# Showing the results of who is more likely to be at risk for heart disease
if male_heart_attacks > female_heart_attacks:
    print("Males are more likely to be at risk for heart disease.")
elif female_heart_attacks > male_heart_attacks:
    print("Females are more likely to be at risk for heart disease.")
else:
    print("Males and females are equally likely to be at risk for heart disease.")



+------+------+
|   Sex| count|
+------+------+
|Female|127811|
|  Male|118211|
+------+------+

+------+-----------------+
|   Sex|TotalHeartAttacks|
+------+-----------------+
|Female|             4932|
|  Male|             8503|
+------+-----------------+

Males are more likely to be at risk for heart disease.


In [14]:
# 4. How does age factor in to heart disease risk?

# Creating a temporary view of the DataFrame
heart_disease_df.createOrReplaceTempView("heart_disease")

# Writing a SQL query to calculate the average age for individuals at risk for heart disease
sql_query = """
    SELECT AgeCategory, AVG(HadHeartAttack) * 100 AS AverageHeartDiseaseRisk
    FROM heart_disease
    GROUP BY AgeCategory
    ORDER BY AgeCategory;
"""
# Executing the SQL query
avg_age_at_risk = spark.sql(sql_query)

# Showing the SQL query results
avg_age_at_risk.show()

+---------------+-----------------------+
|    AgeCategory|AverageHeartDiseaseRisk|
+---------------+-----------------------+
|   Age 18 to 24|     0.3810394756896815|
|   Age 25 to 29|    0.42308038527320185|
|   Age 30 to 34|     0.6743593586093212|
|   Age 35 to 39|     0.9991033687716153|
|   Age 40 to 44|     1.3433099628822247|
|   Age 45 to 49|      2.507013669193577|
|   Age 50 to 54|     3.5303570531813393|
|   Age 55 to 59|      5.003599712023038|
|   Age 60 to 64|      5.894461077844312|
|   Age 65 to 69|      7.546310886997934|
|   Age 70 to 74|      9.355452814794669|
|   Age 75 to 79|     11.386193206881341|
|Age 80 or older|     13.616973506960036|
+---------------+-----------------------+



In [15]:
# 5_a. How many of each Race/Ethnicity exsist in the DataFrame?

# Creating a temporary viewof the DataFrame
heart_disease_df.createOrReplaceTempView("Race_Ethnicity")

# Writing a SQL query to calculate the count of the different Race/Ethnicities in the DataFrame
sql_query = """
    SELECT RaceEthnicityCategory, COUNT(*) AS Count
    FROM Race_Ethnicity
    GROUP BY RaceEthnicityCategory;
"""
# Executing the SQL query
Race_Ethnicity_count= spark.sql(sql_query)

# Showing the SQL query results
Race_Ethnicity_count.show()


+---------------------+------+
|RaceEthnicityCategory| Count|
+---------------------+------+
| Black only, Non-H...| 19330|
| Other race only, ...| 12205|
| Multiracial, Non-...|  5581|
|             Hispanic| 22570|
| White only, Non-H...|186336|
+---------------------+------+



In [16]:
# 5_b. How does Race/Ethnicity factor in to heart disease risk?

# Writing a SQL query to show heart disease prevalence according to Race/Ethnicity on average
sql_query = """
    SELECT
        RaceEthnicityCategory,
        AVG(CASE WHEN HadHeartAttack = 1 THEN 1 ELSE 0 END) * 100 AS HeartDiseasePrevalence
    FROM
        Race_Ethnicity
    GROUP BY
        RaceEthnicityCategory
    ORDER BY
        RaceEthnicityCategory;
"""

# Executing the SQL query
Race_Ethnicity_at_risk = spark.sql(sql_query)

# Showing the SQL query results
Race_Ethnicity_at_risk.show()



+---------------------+----------------------+
|RaceEthnicityCategory|HeartDiseasePrevalence|
+---------------------+----------------------+
| Black only, Non-H...|    4.5990688049663735|
|             Hispanic|    3.8059370846256093|
| Multiracial, Non-...|     6.092098190288478|
| Other race only, ...|     4.842277755018435|
| White only, Non-H...|    5.7723681950884425|
+---------------------+----------------------+



In [17]:
# 6. How do the BMI(Body Mass Index) values impact risk for heat disease?

# Creating a temporary viewof the DataFrame
heart_disease_df.createOrReplaceTempView("BMI")

sql_query = """
    SELECT
        AVG(BMI) AS AverageBMI,
        AVG(HadHeartAttack) AS HeartDiseasePrevalence
    FROM
        BMI
    GROUP BY
        HadHeartAttack;

"""

# Executing the SQL query
bmi_numb_at_risk = spark.sql(sql_query)

# Showing the SQL query results
bmi_numb_at_risk.show()


+------------------+----------------------+
|        AverageBMI|HeartDiseasePrevalence|
+------------------+----------------------+
|29.492435429847518|                   1.0|
|28.620521482282513|                   0.0|
+------------------+----------------------+

